## Data Loading

In [1]:
# import libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Load Data

In [2]:
# Local authority traffic figures give the total volume (vehicle miles) of traffic across each local authority for the whole year.
# pd_local_points = pd.read_csv('../data/uk_data/local_authority_traffic.csv')

In [3]:
# pd_local_points.columns

### Initial Inspection - UK Daaset

In [4]:
# Vehicle counts recorded on major and minor roads
pd_raw_count = pd.read_csv('../data/uk_data/dft_traffic_counts_raw_counts.csv')

/var/folders/_z/nz5spzqj2rl11n5lc_k1cwwc0000gn/T/ipykernel_97095/2200544698.py:2: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  pd_raw_count = pd.read_csv('../data/uk_data/dft_traffic_counts_raw_counts.csv')


In [5]:
pd_raw_count.head()

,count_point_id,direction_of_travel,year,count_date,hour,region_id,region_name,region_ons_code,local_authority_id,local_authority_name,...,buses_and_coaches,LGVs,HGVs_2_rigid_axle,HGVs_3_rigid_axle,HGVs_4_or_more_rigid_axle,HGVs_3_or_4_articulated_axle,HGVs_5_articulated_axle,HGVs_6_articulated_axle,all_HGVs,all_motor_vehicles
0,51,N,2004,2004-05-21,7,1,South West,E12000009,1,Isles of Scilly,...,1.0,13,0.0,0.0,0.0,0.0,0,0.0,0.0,19.0
1,51,N,2004,2004-05-21,8,1,South West,E12000009,1,Isles of Scilly,...,2.0,13,7.0,7.0,0.0,0.0,0,0.0,14.0,44.0
2,51,N,2004,2004-05-21,9,1,South West,E12000009,1,Isles of Scilly,...,5.0,17,5.0,0.0,0.0,0.0,0,0.0,5.0,45.0
3,51,N,2004,2004-05-21,10,1,South West,E12000009,1,Isles of Scilly,...,4.0,4,4.0,0.0,0.0,0.0,0,0.0,4.0,41.0
4,51,N,2004,2004-05-21,11,1,South West,E12000009,1,Isles of Scilly,...,3.0,14,6.0,0.0,0.0,0.0,0,0.0,6.0,41.0


In [6]:
print(f"The shape is :\n{pd_raw_count.shape}")

The shape is :
(4960860, 35)


In [7]:
pd_raw_count_east = pd_raw_count[pd_raw_count['region_name'] == 'East of England'].copy()

In [8]:
pd_raw_count_east.head()

,count_point_id,direction_of_travel,year,count_date,hour,region_id,region_name,region_ons_code,local_authority_id,local_authority_name,...,buses_and_coaches,LGVs,HGVs_2_rigid_axle,HGVs_3_rigid_axle,HGVs_4_or_more_rigid_axle,HGVs_3_or_4_articulated_axle,HGVs_5_articulated_axle,HGVs_6_articulated_axle,all_HGVs,all_motor_vehicles
30144,6003,N,2000,2000-03-31,7,7,East of England,E12000006,208,Bedfordshire,...,5.0,513,171.0,40.0,36.0,63.0,223,145.0,678.0,4006.0
30145,6003,N,2000,2000-03-31,8,7,East of England,E12000006,208,Bedfordshire,...,11.0,436,203.0,17.0,11.0,60.0,180,119.0,590.0,4142.0
30146,6003,N,2000,2000-03-31,9,7,East of England,E12000006,208,Bedfordshire,...,28.0,372,226.0,24.0,17.0,85.0,212,117.0,681.0,3325.0
30147,6003,N,2000,2000-03-31,10,7,East of England,E12000006,208,Bedfordshire,...,26.0,457,202.0,26.0,22.0,80.0,262,157.0,749.0,3500.0
30148,6003,N,2000,2000-03-31,11,7,East of England,E12000006,208,Bedfordshire,...,41.0,492,220.0,24.0,17.0,77.0,218,172.0,728.0,3693.0


In [9]:
pd_raw_count_east.shape

(550056, 35)

In [10]:
pd_raw_count_east.info()

<class 'pandas.core.frame.DataFrame'>
Index: 550056 entries, 30144 to 4960547
Data columns (total 35 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   count_point_id                550056 non-null  int64  
 1   direction_of_travel           550056 non-null  object 
 2   year                          550056 non-null  int64  
 3   count_date                    550056 non-null  object 
 4   hour                          550056 non-null  int64  
 5   region_id                     550056 non-null  int64  
 6   region_name                   550056 non-null  object 
 7   region_ons_code               550056 non-null  object 
 8   local_authority_id            550056 non-null  int64  
 9   local_authority_name          550056 non-null  object 
 10  local_authority_code          550056 non-null  object 
 11  road_name                     550056 non-null  object 
 12  road_category                 550056 non-nul

In [11]:
pd_raw_count_east.columns

Index(['count_point_id', 'direction_of_travel', 'year', 'count_date', 'hour',
       'region_id', 'region_name', 'region_ons_code', 'local_authority_id',
       'local_authority_name', 'local_authority_code', 'road_name',
       'road_category', 'road_type', 'start_junction_road_name',
       'end_junction_road_name', 'easting', 'northing', 'latitude',
       'longitude', 'link_length_km', 'link_length_miles', 'pedal_cycles',
       'two_wheeled_motor_vehicles', 'cars_and_taxis', 'buses_and_coaches',
       'LGVs', 'HGVs_2_rigid_axle', 'HGVs_3_rigid_axle',
       'HGVs_4_or_more_rigid_axle', 'HGVs_3_or_4_articulated_axle',
       'HGVs_5_articulated_axle', 'HGVs_6_articulated_axle', 'all_HGVs',
       'all_motor_vehicles'],
      dtype='object')

In [12]:
pd_raw_count['region_name'].value_counts()

region_name
South East                  792132
North West                  579684
East of England             550056
South West                  493632
West Midlands               480876
Yorkshire and the Humber    427968
London                      403560
East Midlands               398892
Scotland                    306096
Wales                       294432
North East                  233532
Name: count, dtype: int64

In [13]:
pd_raw_count.value_counts()

count_point_id  direction_of_travel  year  count_date  hour  region_id  region_name    region_ons_code  local_authority_id  local_authority_name  local_authority_code  road_name  road_category  road_type  start_junction_road_name  end_junction_road_name   easting  northing  latitude   longitude  link_length_km  link_length_miles  pedal_cycles  two_wheeled_motor_vehicles  cars_and_taxis  buses_and_coaches  LGVs  HGVs_2_rigid_axle  HGVs_3_rigid_axle  HGVs_4_or_more_rigid_axle  HGVs_3_or_4_articulated_axle  HGVs_5_articulated_axle  HGVs_6_articulated_axle  all_HGVs  all_motor_vehicles
51              E                    2020  2020-10-02  7     1          South West     E12000009        1                   Isles of Scilly       E06000053             A3111      PA             Major      Pierhead, Hugh Town       A3112                    90173    10641     49.915021  -6.317073  0.3             0.19               0             0                           7.0             0.0                1 

### Initial Inspection - East England Dataset

In [14]:
east_england = pd.read_csv("/Users/emillianlandu/capstone-Emillia-rosette/data/dft_rawcount_region_id_7.csv")
east_england.head()

/var/folders/_z/nz5spzqj2rl11n5lc_k1cwwc0000gn/T/ipykernel_97095/3447678523.py:1: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  east_england = pd.read_csv("/Users/emillianlandu/capstone-Emillia-rosette/data/dft_rawcount_region_id_7.csv")


,count_point_id,direction_of_travel,year,count_date,hour,region_id,region_name,local_authority_id,local_authority_name,road_name,...,buses_and_coaches,lgvs,hgvs_2_rigid_axle,hgvs_3_rigid_axle,hgvs_4_or_more_rigid_axle,hgvs_3_or_4_articulated_axle,hgvs_5_articulated_axle,hgvs_6_articulated_axle,all_hgvs,all_motor_vehicles
0,6003,N,2000,2000-03-31,7,7,East of England,208,Bedfordshire,M1,...,5,513,171.0,40,36,63,223,145,678.0,4006.0
1,6003,N,2000,2000-03-31,8,7,East of England,208,Bedfordshire,M1,...,11,436,203.0,17,11,60,180,119,590.0,4142.0
2,6003,N,2000,2000-03-31,9,7,East of England,208,Bedfordshire,M1,...,28,372,226.0,24,17,85,212,117,681.0,3325.0
3,6003,N,2000,2000-03-31,10,7,East of England,208,Bedfordshire,M1,...,26,457,202.0,26,22,80,262,157,749.0,3500.0
4,6003,N,2000,2000-03-31,11,7,East of England,208,Bedfordshire,M1,...,41,492,220.0,24,17,77,218,172,728.0,3693.0


In [15]:
east_england.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550056 entries, 0 to 550055
Data columns (total 32 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   count_point_id                550056 non-null  int64  
 1   direction_of_travel           550056 non-null  object 
 2   year                          550056 non-null  int64  
 3   count_date                    550056 non-null  object 
 4   hour                          550056 non-null  int64  
 5   region_id                     550056 non-null  int64  
 6   region_name                   550056 non-null  object 
 7   local_authority_id            550056 non-null  int64  
 8   local_authority_name          550056 non-null  object 
 9   road_name                     550056 non-null  object 
 10  road_type                     550056 non-null  object 
 11  start_junction_road_name      220716 non-null  object 
 12  end_junction_road_name        220716 non-nul

In [16]:
east_england.shape

(550056, 32)

In [17]:
east_england.columns

Index(['count_point_id', 'direction_of_travel', 'year', 'count_date', 'hour',
       'region_id', 'region_name', 'local_authority_id',
       'local_authority_name', 'road_name', 'road_type',
       'start_junction_road_name', 'end_junction_road_name', 'easting',
       'northing', 'latitude', 'longitude', 'link_length_km',
       'link_length_miles', 'pedal_cycles', 'two_wheeled_motor_vehicles',
       'cars_and_taxis', 'buses_and_coaches', 'lgvs', 'hgvs_2_rigid_axle',
       'hgvs_3_rigid_axle', 'hgvs_4_or_more_rigid_axle',
       'hgvs_3_or_4_articulated_axle', 'hgvs_5_articulated_axle',
       'hgvs_6_articulated_axle', 'all_hgvs', 'all_motor_vehicles'],
      dtype='object')

In [18]:
east_england['local_authority_name'].value_counts()

local_authority_name
Essex                   134436
Hertfordshire           104760
Norfolk                  94752
Cambridgeshire           62088
Suffolk                  61572
Peterborough             22416
Thurrock                 16224
Bedfordshire             14844
Central Bedfordshire     12276
Luton                     9828
Southend-on-Sea           8676
Bedford                   8184
Name: count, dtype: int64

In [19]:
east_england['direction_of_travel'].value_counts()

direction_of_travel
N    145584
S    145332
E    129684
W    129372
s        48
w        24
C        12
Name: count, dtype: int64

In [20]:
east_england['road_name'].value_counts()

road_name
U        163716
C         93972
A14       12024
A12       11436
A1         7524
          ...  
B1440        24
B1441        24
A282         24
B1155        24
A1341        24
Name: count, Length: 360, dtype: int64

In [21]:
east_england['road_type'].value_counts()

road_type
Minor    329112
Major    220944
Name: count, dtype: int64

### Data Formatting

In [25]:
east_england.columns

Index(['count_point_id', 'direction_of_travel', 'year', 'count_date', 'hour',
       'region_id', 'region_name', 'local_authority_id',
       'local_authority_name', 'road_name', 'road_type',
       'start_junction_road_name', 'end_junction_road_name', 'easting',
       'northing', 'latitude', 'longitude', 'link_length_km',
       'link_length_miles', 'pedal_cycles', 'two_wheeled_motor_vehicles',
       'cars_and_taxis', 'buses_and_coaches', 'lgvs', 'hgvs_2_rigid_axle',
       'hgvs_3_rigid_axle', 'hgvs_4_or_more_rigid_axle',
       'hgvs_3_or_4_articulated_axle', 'hgvs_5_articulated_axle',
       'hgvs_6_articulated_axle', 'all_hgvs', 'all_motor_vehicles'],
      dtype='object')

In [26]:
# conver `count_date` to datetime type
east_england['count_date'] =  pd.to_datetime(pd_raw_count['count_date'])

In [27]:
# Check data types
print("\nData types in pd_raw_count:")
print(east_england.dtypes)


Data types in pd_raw_count:
count_point_id                           int64
direction_of_travel                     object
year                                     int64
count_date                      datetime64[ns]
hour                                     int64
region_id                                int64
region_name                             object
local_authority_id                       int64
local_authority_name                    object
road_name                               object
road_type                               object
start_junction_road_name                object
end_junction_road_name                  object
easting                                  int64
northing                                 int64
latitude                               float64
longitude                              float64
link_length_km                         float64
link_length_miles                      float64
pedal_cycles                             int64
two_wheeled_motor_vehicles     

In [28]:
# Extract Date Components 
east_england['Month'] = east_england['count_date'].dt.month
east_england['Day_of_Week'] = east_england['count_date'].dt.day_name()
east_england['Day'] = east_england['count_date'].dt.day

In [29]:
east_england['count_date'].head()

0   2004-05-21
1   2004-05-21
2   2004-05-21
3   2004-05-21
4   2004-05-21
Name: count_date, dtype: datetime64[ns]

In [30]:
# Create a 'timestamp' column by combining 'count_date' and 'hour'
east_england['timestamp'] = pd.to_datetime(east_england['count_date']) + pd.to_timedelta(east_england['hour'], unit='h')

In [31]:
east_england.head()

,count_point_id,direction_of_travel,year,count_date,hour,region_id,region_name,local_authority_id,local_authority_name,road_name,...,hgvs_4_or_more_rigid_axle,hgvs_3_or_4_articulated_axle,hgvs_5_articulated_axle,hgvs_6_articulated_axle,all_hgvs,all_motor_vehicles,Month,Day_of_Week,Day,timestamp
0,6003,N,2000,2004-05-21,7,7,East of England,208,Bedfordshire,M1,...,36,63,223,145,678.0,4006.0,5,Friday,21,2004-05-21 07:00:00
1,6003,N,2000,2004-05-21,8,7,East of England,208,Bedfordshire,M1,...,11,60,180,119,590.0,4142.0,5,Friday,21,2004-05-21 08:00:00
2,6003,N,2000,2004-05-21,9,7,East of England,208,Bedfordshire,M1,...,17,85,212,117,681.0,3325.0,5,Friday,21,2004-05-21 09:00:00
3,6003,N,2000,2004-05-21,10,7,East of England,208,Bedfordshire,M1,...,22,80,262,157,749.0,3500.0,5,Friday,21,2004-05-21 10:00:00
4,6003,N,2000,2004-05-21,11,7,East of England,208,Bedfordshire,M1,...,17,77,218,172,728.0,3693.0,5,Friday,21,2004-05-21 11:00:00


In [32]:
east_england.shape

(550056, 36)

In [33]:
columns_to_drop = [
    'year', 'region_id', 'region_name', 'local_authority_id', 'road_name', 
    'easting', 'northing', 'start_junction_road_name', 'end_junction_road_name', 
    'link_length_km', 'link_length_miles', 'latitude', 'longitude', 'pedal_cycles', 
    'hgvs_2_rigid_axle', 'hgvs_3_rigid_axle', 'hgvs_4_or_more_rigid_axle', 
    'hgvs_3_or_4_articulated_axle', 'hgvs_5_articulated_axle', 'hgvs_6_articulated_axle', 
    'all_hgvs', 'buses_and_coaches', 'cars_and_taxis', 'two_wheeled_motor_vehicles', 
    'lgvs'
]

east_england_data = east_england.drop(columns=columns_to_drop)

In [34]:
east_england_data.sample(10)

,count_point_id,direction_of_travel,count_date,hour,local_authority_name,road_type,all_motor_vehicles,Month,Day_of_Week,Day,timestamp
119682,46514,E,2006-06-26,13,Norfolk,Major,533.0,6,Monday,26,2006-06-26 13:00:00
477345,950917,S,2000-10-02,16,Hertfordshire,Minor,131.0,10,Monday,2,2000-10-02 16:00:00
343457,940981,W,2004-03-12,12,Cambridgeshire,Minor,125.0,3,Friday,12,2004-03-12 12:00:00
526423,969152,W,2015-04-28,14,Norfolk,Minor,79.0,4,Tuesday,28,2015-04-28 14:00:00
293417,820105,N,2002-06-14,13,Suffolk,Minor,32.0,6,Friday,14,2002-06-14 13:00:00
108583,38497,S,2003-03-14,14,Hertfordshire,Major,1492.0,3,Friday,14,2003-03-14 14:00:00
345314,941001,N,2019-05-17,9,Cambridgeshire,Minor,26.0,5,Friday,17,2019-05-17 09:00:00
513257,965563,N,2010-05-17,12,Bedfordshire,Minor,259.0,5,Monday,17,2010-05-17 12:00:00
125884,47252,W,2002-10-11,11,Cambridgeshire,Major,455.0,10,Friday,11,2002-10-11 11:00:00
111303,38817,E,2010-06-07,10,Norfolk,Major,1526.0,6,Monday,7,2010-06-07 10:00:00


### Check Data Duplicates

**Notes**:

- No duplicates found in either dataset as data already aggrated

In [35]:
duplicates = east_england_data.duplicated().value_counts()
print(duplicates) 

False    549471
True        585
Name: count, dtype: int64


In [36]:
east_england_data_cleaned = east_england_data.drop_duplicates()

In [37]:
east_england_data_cleaned.duplicated()

0         False
1         False
2         False
3         False
4         False
          ...  
550051    False
550052    False
550053    False
550054    False
550055    False
Length: 549471, dtype: bool

### Check for Nulls

In [38]:
print("\nMissing values in pd_raw_count:")
print((east_england_data.isna().sum() * 100 / east_england_data.shape[0]).round(2))


Missing values in pd_raw_count:
count_point_id          0.0
direction_of_travel     0.0
count_date              0.0
hour                    0.0
local_authority_name    0.0
road_type               0.0
all_motor_vehicles      0.0
Month                   0.0
Day_of_Week             0.0
Day                     0.0
timestamp               0.0
dtype: float64


### Final Investigation

In [39]:
east_england_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550056 entries, 0 to 550055
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   count_point_id        550056 non-null  int64         
 1   direction_of_travel   550056 non-null  object        
 2   count_date            550056 non-null  datetime64[ns]
 3   hour                  550056 non-null  int64         
 4   local_authority_name  550056 non-null  object        
 5   road_type             550056 non-null  object        
 6   all_motor_vehicles    550054 non-null  float64       
 7   Month                 550056 non-null  int32         
 8   Day_of_Week           550056 non-null  object        
 9   Day                   550056 non-null  int32         
 10  timestamp             550056 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int32(2), int64(2), object(4)
memory usage: 42.0+ MB


In [40]:
east_england_data_cleaned.shape

(549471, 11)

#### Save cleaned data to a new CSV

In [41]:
east_england_data_cleaned.to_csv('../data/east_england_data_revised_cleaned.csv', index=False)